In [1]:
from json import loads
from requests import get
from time import sleep
from winotify import Notification

In [2]:
ORIGINS = ['11320000'] # Origin City Code
DESTINATIONS = ['54450000', '54470000', '54310000', '54340000'] # Destination City Code
DATES = ['1402-01-30', '1402-01-31']
TRIES = 1000
SLEEP = 60 # in seconds

In [3]:
t = 0
fullData = {}
notifications = []

while t < TRIES:
    print(f'try number {t + 1}')
    for origin in ORIGINS:
        for destination in DESTINATIONS:
            for date in DATES:
                url = f'https://safar724.com/bus/getservices?origin={origin}&destination={destination}&date={date}'
                fullData[date] = loads(get(url).content)

                originEnglishName = fullData[date]["OriginEnglishName"]
                destinationEnglishName = fullData[date]["DestinationEnglishName"]

                for serviceData in fullData[date]['Items']:
                    availableSeatCount = serviceData['AvailableSeatCount']
                    serviceId = serviceData["ID"]
                    departureTime = serviceData['DepartureTime']

                    if availableSeatCount != 0:
                        notificationMessage = f'{originEnglishName} to {destinationEnglishName} in {date} {departureTime} with {availableSeatCount} available seats'
                        notificationUrl = f'https://safar724.com/bus/{originEnglishName}-{destinationEnglishName}?date={date}/#/service:{serviceId}-{destination}'
                        notification = f'{notificationMessage} {notificationUrl}'

                        if notification not in notifications:
                            notifications.append(notification)
                            toast = Notification(app_id = 'Ticket Alert', title = 'New ticket', msg = notificationMessage)
                            toast.add_actions(label = 'Go to ticket', launch = notificationUrl)
                            toast.show()
                            print(notification)
                            sleep(1)
                        else:
                            print(notification, '[duplicate]')

    t += 1
    sleep(SLEEP)

try number 1
tehran to lahijan in 1402-01-30 07:00 with 14 available seats https://safar724.com/bus/tehran-lahijan?date=1402-01-30/#/service:26404418-54450000
tehran to lahijan in 1402-01-30 09:30 with 13 available seats https://safar724.com/bus/tehran-lahijan?date=1402-01-30/#/service:26404689-54450000
tehran to langarud in 1402-01-30 09:30 with 13 available seats https://safar724.com/bus/tehran-langarud?date=1402-01-30/#/service:26404689-54470000
tehran to rasht in 1402-01-30 09:15 with 14 available seats https://safar724.com/bus/tehran-rasht?date=1402-01-30/#/service:26404882-54310000
try number 2
tehran to lahijan in 1402-01-30 07:00 with 14 available seats https://safar724.com/bus/tehran-lahijan?date=1402-01-30/#/service:26404418-54450000 [duplicate]
tehran to lahijan in 1402-01-30 09:30 with 11 available seats https://safar724.com/bus/tehran-lahijan?date=1402-01-30/#/service:26404689-54450000
tehran to langarud in 1402-01-30 09:30 with 11 available seats https://safar724.com/bus/